# Numerical Optimization | Project 2 | Group 11

In [1]:
import numpy as np
from prettytable import PrettyTable
from scipy.optimize import minimize
import time

#pip install prettytable in case you don't have it

# 1) Implementation of the Simplex Algorithm for LP

In [2]:
def SimplexAlgo(A, b, c, maximize=True, stop_at=1e-6):

    m, n = A.shape
    bas = list(range(n, n + m))
    tab = np.block([
        [A, np.eye(m), b.reshape(-1, 1)],
        [c, np.zeros((1, m + 1))]
    ])

    def iterate(tab, bas, iterations):
        while not np.all(tab[-1, :-1] >= -stop_at):
            pivot_column = np.argmin(tab[-1, :-1]) if maximize else np.argmax(tab[-1, :-1])
            if all(tab[:-1, pivot_column] <= 0):
                return np.zeros(n), iterations, "Simplex did not converge!"
            
            pivot_row = np.argmin(np.where(tab[:-1, pivot_column] > 0, np.divide(tab[:-1, -1], (tab[:-1, pivot_column] + 1e-10)), np.inf))
            tab[pivot_row, :] = np.divide(tab[pivot_row, :], tab[pivot_row, pivot_column])

            for row in range(len(tab)):
                if row != pivot_row:
                    tab[row, :] = tab[row, :] - tab[row, pivot_column] * tab[pivot_row, :]
            bas[pivot_row] = pivot_column
            iterations += 1

        final_iterate = np.zeros(n)
        for row_idx in range(m):
            bas_col = bas[row_idx]
            if bas_col < n:
                final_iterate[bas_col] = tab[row_idx, -1]

        return final_iterate, iterations, "Optimal solution found"

    result, num_iterations, status = iterate(tab, bas, 0)

    if maximize:
        return result, num_iterations, status
    else:
        return -result, num_iterations, status


# 2) 2-variable and 10-variable LP problems

In [3]:
LP_problems_2vars = [
    {'A': np.array([[2, 1], [1, 3], [-1, -1]]), 'b': np.array([8, 9, -1]), 'c': np.array([-3, -2]), 'feas_strt_point': np.array([3, 2]), 'non-feas_strt_point': np.array([7, 2])},
    {'A': np.array([[1, 2], [2, 1], [-1, -2]]), 'b': np.array([5, 6, -3]), 'c': np.array([-4, -1]), 'feas_strt_point': np.array([2.3, 1.3]), 'non-feas_strt_point': np.array([1, 3])},
    {'A': np.array([[3, 2], [1, 2], [-2, -1]]), 'b': np.array([12, 8, -4]), 'c': np.array([-2, -3]), 'feas_strt_point': np.array([2, 3]), 'non-feas_strt_point': np.array([2, -0.5])},
    {'A': np.array([[1, 1], [1, -1], [-1, -1]]), 'b': np.array([4, 1, -2]), 'c': np.array([-1, -2]), 'feas_strt_point': np.array([2.5, 1.5]), 'non-feas_strt_point': np.array([0.5, 1])},
    {'A': np.array([[2, 3], [3, 1], [-1, -2]]), 'b': np.array([7, 8, -3]), 'c': np.array([-3, -4]), 'feas_strt_point': np.array([2.4, 0.7]), 'non-feas_strt_point': np.array([0.5, 1.5])}
]

LP_problems_10vars = [
    {
        'A': np.array([
            [1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
            [2, 1, 0, 1, 2, 1, 0, 1, 2, 1],
            [0, 2, 1, 2, 1, 3, 1, 2, 1, 0],
            [1, 2, 0, 3, 1, 2, 0, 1, 4, 1],
            [2, 1, 0, 1, 3, 1, 1, 2, 1, 2],
            [1, 1, 0, 1, 1, 2, 0, 0, 1, 0],
            [0, 0, 3, 3, 0, 1, 2, 0, 0, 1],
            [0, 1, 1, 1, 2, 1, 1, 1, 1, 1],
            [1, 1, 0, 0, 1, 0, 2, 1, 0, 0],
            [0, 3, 1, 0, 1, 2, 1, 0, 0, 1]
        ]),
        'b': np.array([1, 5, 3, 4, 6, 9, 7, 2, 10, 11]),
        'c': np.array([-1, -8, -3, -4, -5, -6, -7, -2, -9, -5]),
        'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    },
    {
        'A': np.array([
            [1, 2, 1, 1, 2, 1, 1, 2, 0, 1],
            [2, 0, 2, 2, 0, 3, 2, 1, 2, 2],
            [3, 1, 2, 1, 0, 1, 3, 2, 0, 0],
            [1, 2, 3, 1, 1, 2, 3, 0, 2, 1],
            [2, 1, 1, 2, 3, 1, 2, 1, 1, 2],
            [1, 3, 2, 0, 0, 3, 2, 0, 2, 1],
            [2, 1, 3, 2, 1, 1, 3, 2, 1, 3],
            [1, 2, 0, 0, 2, 1, 0, 2, 1, 2],
            [0, 1, 2, 0, 3, 1, 2, 3, 1, 1],
            [1, 3, 1, 2, 1, 3, 1, 2, 1, 1]
        ]),
        'b': np.array([12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
        'c': np.array([-2, -1, -4, -3, -6, -5, -8, -7, -10, -9]),
        'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    },
    {
        'A': np.array([
            [1, 1, 1, 0, 1, 1, 1, 2, 1, 1],
            [0, 1, 2, 1, 2, 0, 1, 1, 2, 2],
            [3, 2, 0, 2, 1, 1, 0, 1, 2, 1],
            [1, 0, 1, 2, 3, 2, 1, 1, 3, 2],
            [2, 0, 1, 1, 2, 1, 0, 2, 1, 0],
            [1, 1, 2, 3, 0, 1, 2, 1, 1, 0],
            [1, 0, 1, 2, 1, 0, 3, 0, 1, 1],
            [2, 1, 2, 1, 2, 1, 0, 3, 1, 2],
            [1, 2, 0, 2, 0, 0, 1, 0, 2, 1],
            [3, 1, 1, 1, 0, 2, 1, 2, 1, 0]
        ]),
        'b': np.array([20, 18, 16, 24, 22, 28, 26, 14, 12, 30]),
        'c': np.array([-3, -1, -5, -2, -4, -6, -8, -7, -9, -10]),
        'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    },
    {
        'A': np.array([
            [2, 1, 2, 1, 2, 1, 2, 0, 2, 1],
            [1, 2, 1, 3, 0, 2, 1, 1, 1, 2],
            [1, 2, 1, 0, 4, 1, 2, 0, 0, 1],
            [1, 0, 2, 1, 2, 2, 1, 1, 0, 2],
            [2, 1, 3, 0, 1, 3, 2, 1, 0, 1],
            [1, 1, 2, 1, 3, 2, 1, 1, 0, 1],
            [2, 3, 1, 0, 3, 2, 1, 2, 0, 2],
            [1, 1, 3, 2, 1, 3, 2, 1, 1, 2],
            [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
            [1, 2, 3, 1, 0, 1, 2, 0, 1, 1]
        ]),
        'b': np.array([14, 16, 18, 20, 22, 24, 36, 40, 44, 48]),
        'c': np.array([-2, -3, -1, -4, -5, -6, -7, -8, -9, -10]),
        'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    },
    {
        'A': np.array([
            [0, 2, 1, 1, 2, 0, 1, 0, 2, 1],
            [2, 1, 0, 2, 1, 0, 0, 1, 0, 1],
            [0, 1, 2, 1, 0, 1, 2, 1, 1, 2],
            [1, 0, 1, 2, 0, 2, 1, 1, 2, 1],
            [2, 1, 0, 2, 1, 0, 1, 1, 0, 2],
            [1, 2, 1, 3, 1, 1, 2, 1, 0, 2],
            [0, 1, 0, 1, 1, 2, 1, 3, 2, 1],
            [1, 3, 1, 1, 0, 0, 1, 2, 0, 2],
            [2, 1, 2, 0, 1, 0, 2, 1, 0, 1],
            [0, 1, 1, 2, 1, 1, 1, 3, 1, 2]
        ]),
        'b': np.array([15, 27, 19, 31, 21, 23, 17, 25, 29, 13]),
        'c': np.array([-1, -24, -2, -15, -3, -16, -7, -28, -9, -10]),
        'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    }
]

# 3) Runs for 2-variable LP problems

In [4]:
# LP problems with 2 variables
for i, problem in enumerate(LP_problems_2vars):
    A = problem['A']
    b = problem['b']
    c = problem['c']
    feas_point = problem['feas_strt_point']
    nonfeast_point = problem['non-feas_strt_point']

    start_at = time.time()
    iterate_feas, iterations_feas, stop_feas = SimplexAlgo(A, b, c)
    end_at = time.time()
    running_time_feasible = end_at - start_at
    
    start_at = time.time()
    iterate_nonfeas, iterations_nonfeas, stop_nonfeas = SimplexAlgo(A, b, c)
    end_at = time.time()
    running_time_non_feasible = end_at - start_at
    
    table_feas = PrettyTable()
    table_feas.field_names = ["Metric", "Value"]
    table_feas.add_row(["Number of iterations", iterations_feas])
    table_feas.add_row(["Final iterate", iterate_feas])
    table_feas.add_row(["Stopping criteria", stop_feas])
    table_feas.add_row(["Running time", f"{running_time_feasible:.6f} seconds"])
    
    print(f"Problem {i + 1} (Feas. starting point):")
    print(table_feas)
    print("\n")

    table_nonfeas = PrettyTable()
    table_nonfeas.field_names = ["Metric", "Value"]
    table_nonfeas.add_row(["Number of iterations", iterations_nonfeas])
    table_nonfeas.add_row(["Final iterate", iterate_nonfeas])
    table_nonfeas.add_row(["Stopping criteria", stop_nonfeas])
    table_nonfeas.add_row(["Running time", f"{running_time_non_feasible:.6f} seconds"])
    
    print(f"Problem {i + 1} (Non-feas. starting point):")
    print(table_nonfeas)
    print("\n")

Problem 1 (Feas. starting point):
+----------------------+------------------------+
|        Metric        |         Value          |
+----------------------+------------------------+
| Number of iterations |           2            |
|    Final iterate     |        [3. 2.]         |
|  Stopping criteria   | Optimal solution found |
|     Running time     |    0.000558 seconds    |
+----------------------+------------------------+


Problem 1 (Non-feas. starting point):
+----------------------+------------------------+
|        Metric        |         Value          |
+----------------------+------------------------+
| Number of iterations |           2            |
|    Final iterate     |        [3. 2.]         |
|  Stopping criteria   | Optimal solution found |
|     Running time     |    0.000580 seconds    |
+----------------------+------------------------+


Problem 2 (Feas. starting point):
+----------------------+------------------------+
|        Metric        |         Value  

# 4) Runs for 10-variable LP problems

In [5]:
# LP problems with 10 variables
for i, problem in enumerate(LP_problems_10vars):
    A = problem['A']
    b = problem['b']
    c = problem['c']
    feas_point = problem['feas_strt_point']
    nonfeast_point = problem['non-feas_strt_point']
    
    start_at = time.time()
    iterate_feas, iterations_feas, stop_feas = SimplexAlgo(A, b, c)
    end_at = time.time()
    running_time_feasible = end_at- start_at
    
    start_at = time.time()
    iterate_nonfeas, iterations_nonfeas, stop_nonfeas = SimplexAlgo(A, b, c)
    end_at = time.time()
    running_time_non_feasible = end_at - start_at
    
    table_feas = PrettyTable()
    table_feas.field_names = ["Metric", "Value"]
    table_feas.add_row(["Number of iterations", iterations_feas])
    table_feas.add_row(["Final iterate", iterate_feas])
    table_feas.add_row(["Stopping criteria", stop_feas])
    table_feas.add_row(["Running time", f"{running_time_feasible:.6f} seconds"])
    
    print(f"Problem {i + 1} (Feas. starting point):")
    print(table_feas)
    print("\n")

    table_nonfeas = PrettyTable()
    table_nonfeas.field_names = ["Metric", "Value"]
    table_nonfeas.add_row(["Number of iterations", iterations_nonfeas])
    table_nonfeas.add_row(["Final iterate", iterate_nonfeas])
    table_nonfeas.add_row(["Stopping criteria", stop_nonfeas])
    table_nonfeas.add_row(["Running time", f"{running_time_non_feasible:.6f} seconds"])
    
    print(f"Problem {i + 1} (Non-feas. starting point):")
    print(table_nonfeas)
    print("\n")

Problem 1 (Feas. starting point):
+----------------------+-----------------------------------------------------+
|        Metric        |                        Value                        |
+----------------------+-----------------------------------------------------+
| Number of iterations |                          5                          |
|    Final iterate     | [0.   0.   0.5  0.   0.   0.5  0.25 0.   0.75 0.  ] |
|  Stopping criteria   |                Optimal solution found               |
|     Running time     |                   0.000828 seconds                  |
+----------------------+-----------------------------------------------------+


Problem 1 (Non-feas. starting point):
+----------------------+-----------------------------------------------------+
|        Metric        |                        Value                        |
+----------------------+-----------------------------------------------------+
| Number of iterations |                          5      

# 5) Active Set Method for QP Problems | Steepest Descent

In [6]:
def create_qp_matrices(M, y):
    G = M.T @ M
    c = -M.T @ y
    return G, c

class QPProblems:
    def __init__(self):
        self.problems = []
        self._generate_problems()

    def _generate_problems(self):
        for m in range(1, 6):
            n = 2 * m
            M = np.random.randn(m, n)
            norm_M = np.linalg.norm(M, 2)
            y = np.random.randn(m) * norm_M
            self.problems.append((M, y))

    def get_problem(self, index):
        M, y = self.problems[index]
        G, c = create_qp_matrices(M, y)
        A = np.vstack([np.eye(len(c)), -np.eye(len(c))])
        b = np.ones(2 * len(c))
        x0 = np.zeros(len(c))
        return G, c, A, b, x0, M, y

def active_set_method(G, c, A, b, x0):
    def objective(x):
        return 0.5 * np.dot(x, np.dot(G, x)) + np.dot(c, x)

    constraints = [{'type': 'ineq', 'fun': lambda x, A=A[i], b=b[i]: b - np.dot(A, x)} for i in range(len(b))]

    start_time = time.time()
    result = minimize(objective, x0, constraints=constraints, method='SLSQP')
    total_time = time.time() - start_time

    return {
        'solution': result.x,
        'objective_value': result.fun,
        'iterations': result.nit,
        'success': result.success,
        'time': total_time
    }

def steepest_descent(M, y, x0, tol=1e-6, max_iter=1000):
    x = x0
    r = M.T @ (M @ x - y)
    for i in range(max_iter):
        alpha = np.dot(r, r) / np.dot(r, M.T @ (M @ r))
        x = x - alpha * r
        r_new = M.T @ (M @ x - y)
        if np.linalg.norm(r_new) < tol:
            break
        r = r_new
    return x, 0.5 * np.linalg.norm(M @ x - y)**2

In [7]:
qp_problems = QPProblems()

for i in range(len(qp_problems.problems)):
    print(f"QP Problem {i+1}")
    G, c, A, b, x0, M, y = qp_problems.get_problem(i)

    table = PrettyTable()
    table.field_names = ["Run", "Optimal Solution", "Objective Value", "Iterations", "Success", "Time (seconds)"]

    for j in range(3):  # 3 different starting points
        x0 = np.random.randn(len(c))  # Random starting point for variety
        result = active_set_method(G, c, A, b, x0)
        table.add_row([
            f"Run {j+1}",
            np.array2string(result['solution'], precision=6),
            f"{result['objective_value']:.20f}",
            result['iterations'],
            result['success'],
            f"{result['time']:.6f}"
        ])

    print(table)

    # Unconstrained problem using steepest descent
    x0_unconstrained = np.random.randn(M.shape[1])
    solution, obj_value = steepest_descent(M, y, x0_unconstrained)
    
    unconstrained_table = PrettyTable()
    unconstrained_table.field_names = ["Solution", "Objective Value"]
    unconstrained_table.add_row([
        np.array2string(solution, precision=20),
        f"{obj_value:.20f}"
    ])
    print("Unconstrained problem solution:")
    print(unconstrained_table)
    print()

QP Problem 1
+-------+---------------------+-------------------------+------------+---------+----------------+
|  Run  |   Optimal Solution  |     Objective Value     | Iterations | Success | Time (seconds) |
+-------+---------------------+-------------------------+------------+---------+----------------+
| Run 1 | [0.169949 1.      ] | -0.51442495396301546329 |     4      |   True  |    0.011376    |
| Run 2 | [0.644082 0.763161] | -0.51442495396301657351 |     3      |   True  |    0.004477    |
| Run 3 | [0.586178 0.792085] | -0.51442495396301657351 |     3      |   True  |    0.003928    |
+-------+---------------------+-------------------------+------------+---------+----------------+
Unconstrained problem solution:
+-----------------------------------------+------------------------+
|                 Solution                |    Objective Value     |
+-----------------------------------------+------------------------+
| [1.31571838865731   0.4276656986074497] | 0.0000000000000000